In [64]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [ ]:
# LCEL for chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

model = ChatOpenAI(
    model="gpt-4o-mini"
)

# StrOutputParser() ai 의 응답을 내용만 봅아서 볼 수 있음. 원래는 메타 데이터도 포함됨.
chain = prompt | model | StrOutputParser()

print(f"[답변]: {chain.invoke({'topic': 'ice cream'})}")



In [ ]:
# chain 의 stream 기능 써보기
model = ChatOpenAI(
    model="gpt-4o-mini"
)

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

chain = prompt | model

for s in chain.stream({"topic": "ice cream"}):
    print(s.content, end="", flush=True)


Stream 사용 이유
* 빠른 응답 제공: 전체 응답이 생성될 때까지 기다릴 필요 없이, 한 토큰씩 받아볼 수 있음.
* 실시간 처리 가능: UI에서 점진적으로 텍스트를 표시하거나, 스트림 데이터를 다른 곳에서 활용할 수 있음.
* 메모리 효율성: 한 번에 모든 데이터를 로드하지 않기 때문에 메모리 부담이 줄어듦.

In [ ]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "너는 소설 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘."
                'ex(Query: 현대 문학 소설 5개 추천해줘 / 답변: ["aa", "ee", "ee2"]'
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini")
chain = chat_template | model
chain.invoke("사회")

In [13]:
# 부분적으로 프롬포트에 미리 넣어두고 싶은 부분이 있다면?
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

# parser 선언
output_parser = CommaSeparatedListOutputParser()

# csv 파서 작동을 위한 형식 지정 프롬프트 로드
format_instructions = output_parser.get_format_instructions()

# 프롬포트 템플릿의 partial_variables 에 csv 형식 지정 프롬포트 주입
prompt = PromptTemplate(template="List {subject}. {format_instructions}", input_variables=["subject"],
                          partial_variables={"format_instructions": format_instructions}, )

model = ChatOpenAI(model="gpt-4o-mini")
chain = prompt | model | output_parser
chain.invoke({"subject":"한국 현대 문학"})



['김소진', '한강', '김영하', '박완서', '이외수', '조선희', '박범신', '정유정', '신경숙', '최재붕']

In [10]:
from pydantic import Field
from langchain_core.output_parsers import JsonOutputParser
from openai import BaseModel


#jsonOutputParser

class Country(BaseModel):
    continent: str = Field(description="사용자가 물어본 나라가 속한 대륙")
    population: str = Field(description="사용자가 물어본 나라의 인구(int 형식)")


# JsonOutputParser 생성할 때 format_instructions 로 class 넘겨
parser = JsonOutputParser(pydantic_object=Country)

prompt = PromptTemplate(
    template="Answer the user query. in korean. {format_instructions} {query}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

country_query = "태국은 어떤 나라야?"
chain.invoke({"query": country_query})


{'continent': '아시아', 'population': '70199978'}

In [19]:
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template("""다음 한글 문장을 프랑스어로 번역해줘 {sentence}
French sentence: (print from here)""")

model = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

runnable_chain = {"sentence": RunnablePassthrough()} | prompt | model | parser
runnable_chain.invoke({"sentence": "윤지는 아무래도 짱이다."})

'Yunjin est sans doute la meilleure.'

In [38]:
def add_smile(x):
    return x + ":)"

In [26]:
from langchain_core.runnables import RunnableLambda

add_smile = RunnableLambda(add_smile)

In [33]:
prompt_str = "{topic} 의 역사에 대해 세 문장으로 설명해주세요."
prompt = ChatPromptTemplate.from_template(prompt_str)

model = ChatOpenAI(model="gpt-4o-mini")

output_parser = StrOutputParser()

chain = prompt | model | output_parser | add_smile
chain.invoke({"topic": "한국"})

'한국의 역사는 고조선, 백제, 신라, 가야 등의 고대 왕국에서 시작되어 고려와 조선이라는 두 개의 수천 년 역사를 가진 왕조를 거쳐 현대에 이릅니다. 20세기 초 일본의 식민지 통치와 1945년 해방 후의 분단은 한국 역사에서 중요한 전환점이 되었으며, 한국 전쟁(1950-1953)은 남북 간의 갈등을 심화시켰습니다. 이후 한국은 급속한 경제 성장을 이루었으며, 민주화와 문화적 영향력을 세계적으로 확대해 나갔습니다.:)'

In [44]:
from langchain_core.runnables import RunnableParallel

runnable_parallel = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=add_smile
)

runnable_parallel.invoke("하이하이")

{'passed': '하이하이', 'modified': '하이하이:)'}

In [62]:
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# prompt
description_prompt = ChatPromptTemplate.from_template("{topic} 이 무엇을 뜻하는 약자입니까?")
celeb_prompt = ChatPromptTemplate.from_template("{topic} 와 관련된 유명인사는 누구입니까?")
# model
model = ChatOpenAI(model="gpt-4o-mini", max_tokens=128, temperature=0)
# output_parser
output_parser = StrOutputParser()

description_chain = description_prompt | model | output_parser
celeb_chain = celeb_prompt | model | output_parser

map_chain = RunnableParallel(description = description_chain, celeb = celeb_chain)
invoke= map_chain.invoke({"topic": "AI"})
invoke


{'description': 'AI는 "Artificial Intelligence"의 약자로, 한국어로는 "인공지능"이라고 합니다. 인공지능은 컴퓨터 시스템이 인간의 지능을 모방하여 학습, 추론, 문제 해결 등의 작업을 수행할 수 있도록 하는 기술을 의미합니다.',
 'celeb': 'AI와 관련된 유명인사들은 여러 분야에서 활동하고 있으며, 그 중 일부는 다음과 같습니다:\n\n1. **엘론 머스크 (Elon Musk)** - 테슬라와 스페이스X의 CEO로, AI의 발전에 대한 경고와 함께 OpenAI의 공동 창립자이기도 합니다.\n\n2. **제프리 힌튼 (Geoffrey Hinton)** - 딥러닝의 아버지로 불리며, 인공지능 분야에서 중요한 연구를 해온 인물입니다.\n\n3. **얀 르쿤 (Yann LeCun)**'}